In [4]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;

//     H ≔ - J Σ'ᵢⱼ Zᵢ Zⱼ - hZ Σᵢ Zᵢ - hX Σᵢ Xᵢ

operation ApplyZZ(phi : Double, q1 : Qubit, q2 : Qubit) : Unit is Adj + Ctl {
    ApplyWithCA(CNOT(q1, _), Rz(-2.0 * phi, _), q2);
}

operation ApplySingleQubitTerms(nSites: Int, dt: Double, hx: Double[], hz: Double[], qubits : Qubit[]): Unit is Adj + Ctl {
    for (idxSite in 0 .. nSites - 1) {
        Rx((-2.0 * hx[idxSite]) * dt, qubits[idxSite]);
        Rz((-2.0 * hz[idxSite]) * dt, qubits[idxSite]);
        }
}

operation ApplyTwoQubitTerms(nSites: Int, dt: Double, J: Double[][], qubits : Qubit[]): Unit is Adj + Ctl {
    let rows = Length(J);
    let cols = Length(J[0]);
    for (i in 0 .. rows - 1) {
        for (j in 0 .. cols - 1) {
            ApplyZZ(J[i][j] * dt, qubits[i], qubits[j]);
        }
    }
}

operation EvolveSingleTimestep(nSites: Int, dt: Double, hx: Double[], hz: Double[], J: Double[][], qubits : Qubit[]): Unit is Adj + Ctl {
    ApplySingleQubitTerms(nSites, dt, hx, hz, qubits);
    ApplyTwoQubitTerms(nSites, dt, J, qubits);
}

operation EvolveSingleTimestepDummy(nSites: Int, dt: Double, hx: Double[], hz: Double[], J: Double[][]): Unit is Adj + Ctl {
    using (qubits = Qubit[nSites]) {
        ApplySingleQubitTerms(nSites, dt, hx, hz, qubits);
        ApplyTwoQubitTerms(nSites, dt, J, qubits);
    }
}

operation Evolve(initialState: Int[], time: Double, dt: Double, hx: Double[], hz: Double[], J: Double[][]): Result[] {
    let nSites = Length(initialState);
    using (qubits = Qubit[nSites]) {
        for (idxSite in 0 .. nSites - 1) {
            if (initialState[idxSite] == 1) {
                X(qubits[idxSite]);
            }
        }
        let nSteps = Floor(time / dt);
        for (idxIter in 0 .. nSteps - 1) {
            EvolveSingleTimestep(nSites, dt, hx, hz, J, qubits);
        }
        return ForEach(MResetZ, qubits);
    }
}

/snippet_.qs(3,6): warning QS6003: The namespace is already open.
/snippet_.qs(4,6): warning QS6003: The namespace is already open.


ApplySingleQubitTerms, ApplyTwoQubitTerms, ApplyZZ, Evolve, EvolveSingleTimestep, EvolveSingleTimestepDummy